In [1]:
from sqlalchemy import URL, create_engine, text
import streamlit as st

In [6]:
connection_string = URL.create(
    'postgresql',
    username=st.secrets.connections.postgresql.username,
    password=st.secrets.connections.postgresql.password,
    host=st.secrets.connections.postgresql.host,
    database=st.secrets.connections.postgresql.database,
    query={'options':'endpoint=ep-sweet-cloud-a1zh9huf'}
)

In [7]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}, pool_pre_ping=True, pool_recycle=3600)
conn = engine.connect()

## Delete

In [4]:
query = text('SELECT url , COUNT(url) FROM news GROUP BY url HAVING COUNT(url) > 1')
output = conn.execute(query)
rows = output.fetchall()
for row in rows:
    print(row)

In [5]:
query = text('DELETE FROM news a USING news b WHERE a.id < b.id AND a.url = b.url')
conn.execute(query)

In [6]:
query = text('SELECT url , COUNT(url) FROM news GROUP BY url HAVING COUNT(url) > 1')
output = conn.execute(query)
rows = output.fetchall()
for row in rows:
    print(row)

In [8]:
# Remove records that older than 30 days
query = text('DELETE FROM news WHERE date < CURRENT_DATE - INTERVAL \'30 days\'')
conn.execute(query)

In [9]:
conn.commit()

In [8]:
conn.close()

## Crawl and insert

In [7]:
from _crawler import FilterURL, get_all_links, get_content
from _summarizer import create_model, summarize

summarizer_model = create_model('gemini-1.5-pro')

In [8]:
news_sample = get_content('https://e.vnexpress.net/news/news/singapore-allows-online-declaration-for-15-000-cash-transit-4736850.html')
summarized_text_sample = summarize(summarizer_model, full_text=news_sample['text'])

In [9]:
summarized_text_sample = summarized_text_sample.replace("'", "''").replace('"', '')
news_sample['title'] = news_sample['title'].replace('"', '')

In [10]:
query_insert = "INSERT INTO news(url, title, date, author, category, summary) " + \
                f"VALUES('{news_sample['url']}', '{news_sample['title']}', '{news_sample['date']}', '{news_sample['author']}', '{news_sample['category']}', '{summarized_text_sample}')"

In [11]:
query_insert_check = "INSERT INTO NEWS(url, title, date, author, category, summary) " + \
                f"SELECT '{news_sample['url']}', '{news_sample['title']}', '{news_sample['date']}', '{news_sample['author']}', '{news_sample['category']}', '{summarized_text_sample}'" + \
                f"WHERE NOT EXISTS (SELECT 1 FROM NEWS WHERE url = '{news_sample['url']}')"

In [12]:
query_insert_check

"INSERT INTO NEWS(url, title, date, author, category, summary) SELECT 'https://e.vnexpress.net/news/news/singapore-allows-online-declaration-for-15-000-cash-transit-4736850.html', 'Singapore allows online declaration for $15,000 cash transit', '2024-04-21', 'Hoang Phong', 'news', 'Starting May 13, 2023, travelers carrying over S$20,000 (US$14,700) in cash can submit online declarations via the Immigration and Checkpoints Authority (ICA) website or MyICA mobile app.  This replaces the current hard copy requirement.  The online form will be accessible from May 10, 2023, and must be submitted three days prior to arrival or departure.  Non-declaration penalties include up to three years imprisonment and/or a S$50,000 fine.\n'WHERE NOT EXISTS (SELECT 1 FROM NEWS WHERE url = 'https://e.vnexpress.net/news/news/singapore-allows-online-declaration-for-15-000-cash-transit-4736850.html')"

In [13]:
conn.execute(text(query_insert_check))

In [14]:
conn.commit()

In [8]:
output = conn.execute(text("SELECT * FROM news"))

In [9]:
rows = output.fetchall()
for row in rows:
    print(row)

(4006, 'https://e.vnexpress.net/news/news/japan-births-fall-to-new-historic-low-4854885.html', 'Japan births fall to new historic low', datetime.date(2025, 2, 27), 'AFP', 'news', "Japan's 2024 birth rate hit a record low for the ninth consecutive year, declining 5% to 720,988.  Deaths (1.62 million) more than doubled births.  T ... (226 characters truncated) ... ritizes addressing this demographic crisis, though high education costs, economic stagnation, and lifestyle changes contribute to declining births.\n")
(4007, 'https://e.vnexpress.net/news/news/traffic/new-york-city-congestion-program-collected-48-6m-in-january-4854518.html', 'New York City congestion program collected $48.6M in January', datetime.date(2025, 2, 26), 'Reuters', 'news', "NYC's congestion pricing program generated $48.6 million in gross revenue and $37.5 million in net revenue during its first month (January 2024).  Pa ... (456 characters truncated) ...  in London, Singapore, and Sweden.  NYC implemented the progr

In [12]:
conn.commit()

In [4]:
conn.close()